In [6]:
import pandas as pd

df = pd.read_csv("archive/games.csv")
df = df[["black_rating","white_rating","moves","winner"]] #, "opening_eco"]]
df.to_pickle("archive/games.pkl")

In [7]:
df = pd.read_pickle("archive/games.pkl")
print(df.head())
max_rating = max(df["black_rating"].max(), df["white_rating"].max())
df["black_rating_scaled"] = df["black_rating"] / max_rating
df["white_rating_scaled"] = df["white_rating"] / max_rating
df["winner_encoded"] = df["winner"].map({"draw": 0, "white": 1, "black": 2})
df["moves_tokenized"] = df["moves"].apply(lambda x: x.split())
# unique_openings = sorted(list(set(df["opening_eco"])))
unique_moves = sorted(list(set(m for row in df["moves_tokenized"] for m in row)))
# opening_to_idx = {opening: i for i, opening in enumerate(unique_openings)}
move_to_idx = {move: i for i, move in enumerate(unique_moves)}
# df["opening_encoded"] = df["opening_eco"].map(opening_to_idx)
df["moves_encoded"] = df["moves_tokenized"].apply(lambda row: [move_to_idx[m] for m in row])
df.drop(columns=["black_rating", "white_rating", "moves", "winner","moves_tokenized"], inplace=True)
print(df.head())
# print("unique_openings", len(unique_openings))
print("unique_moves", len(unique_moves))
df.to_pickle("archive/games_prepared.pkl")


   black_rating  white_rating  \
0          1191          1500   
1          1261          1322   
2          1500          1496   
3          1454          1439   
4          1469          1523   

                                               moves winner  
0  d4 d5 c4 c6 cxd5 e6 dxe6 fxe6 Nf3 Bb4+ Nc3 Ba5...  white  
1  d4 Nc6 e4 e5 f4 f6 dxe5 fxe5 fxe5 Nxe5 Qd4 Nc6...  black  
2  e4 e5 d3 d6 Be3 c6 Be2 b5 Nd2 a5 a4 c5 axb5 Nc...  white  
3  d4 d5 Nf3 Bf5 Nc3 Nf6 Bf4 Ng4 e3 Nc6 Be2 Qd7 O...  white  
4  e4 e5 Nf3 d6 d4 Nc6 d5 Nb4 a3 Na6 Nc3 Be7 b4 N...  white  
   black_rating_scaled  white_rating_scaled  winner_encoded  \
0             0.437385             0.550863               1   
1             0.463092             0.485494               2   
2             0.550863             0.549394               1   
3             0.533970             0.528461               1   
4             0.539479             0.559310               1   

                                       moves_encod

In [ ]:
# df = pd.read_pickle("archive/games_prepared.pkl")
# df = df.sample(100)
# print(df.head())
# df.to_pickle("archive/games_prepared_sample.pkl")

       black_rating_scaled  white_rating_scaled  winner_encoded  \
18607             0.614396             0.607786               2   
9833              0.570327             0.629453               2   
14071             0.578773             0.554903               0   
19835             0.550863             0.550863               1   
17036             0.510099             0.608520               2   

                                           moves_encoded  
18607  [4217, 4223, 1039, 4154, 4242, 1049, 761, 1295...  
9833   [4152, 4352, 120, 152, 4214, 4094, 1039, 1426,...  
14071  [4152, 4091, 4178, 4223, 93, 1049, 4280, 770, ...  
19835  [4217, 4091, 48, 770, 1039, 1542, 4152, 4130, ...  
17036  [4217, 4220, 1039, 4286, 761, 4154, 4242, 4217...  


In [ ]:
from sklearn.model_selection import train_test_split

seed = 13

train_df, temp_df = train_test_split(df, test_size=0.2, random_state=seed)
test_df, val_df = train_test_split(temp_df, test_size=0.5, random_state=seed)
train_df.to_pickle("archive/games_train.pkl")
test_df.to_pickle("archive/games_test.pkl")
val_df.to_pickle("archive/games_val.pkl")

print(f"Train set size: {len(train_df)}")
print(f"Test set size: {len(test_df)}")
print(f"Validation set size: {len(val_df)}")

Train set size: 16046
Test set size: 2006
Validation set size: 2006


In [ ]:
# dont include the full game!! 